In [1]:
#imports and preprocessing, setting up train_loader
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Unsupervised-concept-centered-prototypical-parts/code

/content/drive/MyDrive/Unsupervised-concept-centered-prototypical-parts/code


In [3]:
import torch
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from torchvision import transforms
from dataset import utils
from dataset.cub2011 import Cub2011
from dataset.CombinedDataLoader import CDL


In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
    gpu_index = 0
else:
    device = torch.device('cpu')
    gpu_index = -1


In [5]:
print(device)

cuda


In [6]:
root = './dataset/data'

In [7]:
#ds_whole = Cub2011(root, train=None, download=True, transform=transforms.ToTensor())

In [8]:
#utils.crop_all_images(root)

In [9]:
transformy = transforms.Compose([
    transforms.Resize((224, 224)), # Resize the image to 224x224
    transforms.ToTensor() # Convert the image to a tensor
])

In [10]:
train_loader1 = CDL(root, train=None, transform=transformy, num_classes=3, images=['cropped', "grayscale", "gaussianBlur", "colorJitter"])

In [11]:
train_loader = torch.utils.data.DataLoader(train_loader1, batch_size=32, shuffle=False, pin_memory=True, drop_last=True)


In [12]:
# declare train_dateset and train_loader

In [13]:
from models.classificationModel import ClassificationModel
from models.classificationTrainer import Trainer

model = ClassificationModel(out_dim=1000, dataset='./dataset/data/CUB_200_2011')



/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
model.backbone.fc = torch.nn.Sequential(#wariant 1
    torch.nn.Linear(in_features=512, out_features=128, bias=True),
    torch.nn.Linear(in_features=128, out_features=20, bias=True),
)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                       last_epoch=-1)

In [16]:
with torch.cuda.device(gpu_index):
    trainer = Trainer(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100,
                      log_dir='test')
    trainer.train(train_loader)

  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:814: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
 73%|███████▎  | 16/22 [03:50<01:26, 14.42s/it]


FileNotFoundError: ignored

In [ ]:
torch.save(model.state_dict(), 'saved_models/test.pth')